#1.Import Libraries

In [ ]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [ ]:
!pip install transformers requests beautifulsoups pandas numpay

ERROR: Could not find a version that satisfies the requirement beautifulsoups (from versions: none)
ERROR: No matching distribution found for beautifulsoups


In [15]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re


In [3]:
# Load novel text
with open('/content/novel.txt', 'r', encoding='utf-8') as file:
    text = file.read()

In [4]:
# Split text into sentences
sentences = re.split(r'(?<=[.!?]) +', text)
print(f"Total sentences: {len(sentences)}")

Total sentences: 582


In [5]:
# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased-finetuned-sst-2-english')
model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased-finetuned-sst-2-english')
model.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [8]:
# Function to analyze sentiment of a sentence
import torch.nn.functional as F
def get_sentiment(sentence):
    inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True, max_length=512)
    outputs = model(**inputs)
    probs = F.softmax(outputs.logits, dim=-1)
    sentiment = torch.argmax(probs).item()
    confidence = probs[0][sentiment].item()
    return sentiment, confidence

In [9]:
# Analyze sentiment for each sentence
sentiment_scores = [(sentence, *get_sentiment(sentence)) for sentence in sentences]

In [10]:
# Prepare data for plotting
sentiments = [score[1] for score in sentiment_scores]
window_size = 50
smoothed_sentiments = [sum(sentiments[i:i+window_size]) / window_size for i in range(len(sentiments) - window_size)]


In [18]:
import plotly.graph_objects as go

# Create a Plotly figure
fig = go.Figure()

# Add a line trace with gradient color and area fill
fig.add_trace(go.Scatter(
    x=list(range(len(smoothed_sentiments))),
    y=smoothed_sentiments,
    mode='lines',
    line=dict(color='royalblue', width=2),
    fill='tozeroy',  # Fills area under the curve
    fillcolor='rgba(65, 105, 225, 0.3)',  # Light blue fill with some transparency
    name='Sentiment'
))

# Highlight specific points (e.g., peaks or major transitions) with markers
# Adjust these points as needed based on your data
highlight_points = [i for i in range(0, len(smoothed_sentiments), 100)]  # every 100th point
highlight_values = [smoothed_sentiments[i] for i in highlight_points]

fig.add_trace(go.Scatter(
    x=highlight_points,
    y=highlight_values,
    mode='markers',
    marker=dict(color='darkorange', size=8, symbol='circle'),
    name='Key Points'
))

# Customize the layout to make it visually appealing
fig.update_layout(
    title={
        'text': 'Sentiment Analysis of the Novel',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    xaxis=dict(
        title='Sentence Number',
        showgrid=False,
        zeroline=False
    ),
    yaxis=dict(
        title='Sentiment Score (Smoothed)',
        showgrid=True,
        zeroline=True,
        zerolinecolor='lightgray'
    ),
    template='plotly_dark',  # Dark theme for a modern look
    plot_bgcolor='rgba(0,0,0,0)',  # Transparent background
    width=1000,
    height=600,
)

# Add a subtle shaded background under the sentiment line for visual depth
fig.update_traces(line_shape='spline')  # Smooth line with spline

# Add annotations for additional clarity (optional)
fig.add_annotation(
    x=len(smoothed_sentiments) // 2,
    y=max(smoothed_sentiments),
    text="Highest Sentiment Point",
    showarrow=True,
    arrowhead=1,
    ax=0,
    ay=-40,
    font=dict(size=12, color="white")
)

fig.show()

